*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [80]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [81]:
# get data files
#!wget "https://cdn.freecodecamp.org/project-data/books/book-crossings.zip"

#!unzip book-crossings.zip

books_filename = '..\data\\fcc_book_recommendation_KNN\BX-Books.csv'
ratings_filename = '..\data\\fcc_book_recommendation_KNN\BX-Book-Ratings.csv'

In [82]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [83]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [84]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [85]:
# add your code here - consider creating a new cell for each section of code

In [86]:
#join dataset  df_ratings df_books
df_rating_Books= pd.merge(df_ratings,df_books, on="isbn")

In [87]:
df_rating_Books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031175 entries, 0 to 1031174
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   user    1031175 non-null  int32  
 1   isbn    1031175 non-null  object 
 2   rating  1031175 non-null  float32
 3   title   1031175 non-null  object 
 4   author  1031174 non-null  object 
dtypes: float32(1), int32(1), object(3)
memory usage: 39.3+ MB


In [88]:
df_rating_Books.head()

,user,isbn,rating,title,author
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
1,2313,034545104X,5.0,Flesh Tones: A Novel,M. J. Rose
2,6543,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
3,8680,034545104X,5.0,Flesh Tones: A Novel,M. J. Rose
4,10314,034545104X,9.0,Flesh Tones: A Novel,M. J. Rose


In [89]:
#check for non value 
df_rating_Books.isna().sum()

user      0
isbn      0
rating    0
title     0
author    1
dtype: int64

In [90]:
#drop non value
df_rating_Books=df_rating_Books.dropna(axis=0,subset=["author"])
df_rating_Books.isna().sum()

user      0
isbn      0
rating    0
title     0
author    0
dtype: int64

In [91]:
#create table to count rating for each book
book_RatingCount= (df_rating_Books.groupby(by=["title"])["rating"].count()
                    .reset_index()
                    .rename(columns={"rating": "totalRating"})
                    [["title","totalRating"]])

book_RatingCount.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241089 entries, 0 to 241088
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   title        241089 non-null  object
 1   totalRating  241089 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.7+ MB


In [92]:
book_RatingCount.head()

,title,totalRating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


#### Remove books with <100 ratings

In [93]:
books = df_rating_Books.groupby("isbn").count().reset_index()

df_rating_Books = df_rating_Books[df_rating_Books['isbn'].isin(books[books["user"] > 100]['isbn'])]

In [94]:
df_rating_Books

,user,isbn,rating,title,author
62,276727,0446520802,0.0,The Notebook,Nicholas Sparks
63,278418,0446520802,0.0,The Notebook,Nicholas Sparks
64,638,0446520802,0.0,The Notebook,Nicholas Sparks
65,3363,0446520802,0.0,The Notebook,Nicholas Sparks
66,7158,0446520802,10.0,The Notebook,Nicholas Sparks
...,...,...,...,...,...
521240,263877,0312971125,0.0,Crazy for You,Jennifer Crusie
521241,265115,0312971125,10.0,Crazy for You,Jennifer Crusie
521242,266056,0312971125,0.0,Crazy for You,Jennifer Crusie
521243,267249,0312971125,0.0,Crazy for You,Jennifer Crusie


#### Remove users with <200 ratings

In [95]:
user = df_rating_Books.groupby("user").count().reset_index()

df_rating_Books = df_rating_Books[df_rating_Books['user'].isin(user[user["isbn"] > 200]['user'])]

In [96]:
df_rating_Books

,user,isbn,rating,title,author
69,11676,0446520802,10.0,The Notebook,Nicholas Sparks
74,23768,0446520802,6.0,The Notebook,Nicholas Sparks
100,102967,0446520802,0.0,The Notebook,Nicholas Sparks
132,185233,0446520802,7.0,The Notebook,Nicholas Sparks
151,230522,0446520802,0.0,The Notebook,Nicholas Sparks
...,...,...,...,...,...
521136,16795,0312971125,0.0,Crazy for You,Jennifer Crusie
521152,76352,0312971125,0.0,Crazy for You,Jennifer Crusie
521153,78783,0312971125,0.0,Crazy for You,Jennifer Crusie
521176,135149,0312971125,7.0,Crazy for You,Jennifer Crusie


#### ---> Prepare dataset for KNN


In [105]:
df_knn = df_rating_Books.pivot_table(index=['isbn'],columns=['user'],values='rating').fillna(0)
sparse_book_ratings=csr_matrix(df_knn)
#sparse_book_ratings
df_knn.head()

user,11676,16795,21014,23768,35859,43246,52584,55492,60244,76352,78783,102967,135149,153662,185233,198711,204864,230522,232131,238120
isbn,,,,,,,,,,,,,,,,,,,,
002542730X,6.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060008032,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060096195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006016848X,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060173289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
model = NearestNeighbors(metric='cosine')
model.fit(sparse_book_ratings)

NearestNeighbors(metric='cosine')

### Build get recommends

In [107]:
title = 'The Queen of the Damned (Vampire Chronicles (Paperback))'
df_knn.loc[title].shape

KeyError: 'The Queen of the Damned (Vampire Chronicles (Paperback))'

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()